In [ ]:
import cv2
from matplotlib import pyplot as plt

import spade
from spade.agent import Agent
from spade.behaviour import TimeoutBehaviour, CyclicBehaviour
from spade.message import Message

from ast import literal_eval

import asyncio
import random
import math

In [ ]:
img = cv2.imread("tokyo.jpg")
#img = cv2.imread("house.jpg")

#RGB to BGR
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

width = img.shape[1]
height = img.shape[0]

In [ ]:
image = []
for row in img:
    row_grey = []
    for col in row:
        pixel = int(col[0] * 114 / 1000 + col[1] * 587 / 1000 + col[2] * 299 / 1000)
        row_grey.append(pixel)
    image.append(row_grey)

In [ ]:
plt.imshow(image, cmap=plt.cm.binary)

In [ ]:
edges = []
for i in range(0, height):
    row = []
    for j in range(0, width):
        row.append(0)
    edges.append(row)

In [ ]:
R = 1
cT = 20 # contrast threshold
lD = 10000 # life duration (steps)

def getSurrounding(location):
    surrounding = []
    for y in range(location[1] - R, location[1] + R + 1):
        for x in range(location[0] - R, location[0] + R + 1):
            if x >= 0 and x < width and y >=0 and y < height and not (x == location[0] and y == location[1]):
                surrounding.append(image[y][x])
    return surrounding

def getSurroundingMatrix(location):
    surrounding = []
    for y in range(location[1] - R, location[1] + R + 1):
        row = []
        for x in range(location[0] - R, location[0] + R + 1):
            if x >= 0 and x < width and y >=0 and y < height:
                row.append(image[y][x])
            else:
                row.append(-1)
        surrounding.append(row)
    return surrounding

def getSurroundingEdges(location):
    surrounding = []
    for y in range(location[1] - R, location[1] + R + 1):
        row = []
        for x in range(location[0] - R, location[0] + R + 1):
            if x >= 0 and x < width and y >=0 and y < height:
                row.append(edges[y][x])
            else:
                row.append(-1)
        surrounding.append(row)
    return surrounding

def getRegionalMean(location):
    surrounding = getSurrounding(location)
    return sum(surrounding) / len(surrounding)

def getStdDev(location):
    surrounding = getSurrounding(location)
    mean = getRegionalMean(location)
    sqSum = 0
    for p in surrounding:
        sqSum += math.pow(p - mean, 2)
    return math.sqrt(sqSum / len(surrounding))

def getNextDirection(location):
    directionsVector = [1, 1, 1, 1, 1, 1, 1, 1]
    if location[1] == 0:
        directionsVector[0] = directionsVector[1] = directionsVector[2] = 0
    if location[0] == 0:
        directionsVector[0] = directionsVector[3] = directionsVector[5] = 0
    if location[1] == height - 1:
        directionsVector[5] = directionsVector[6] = directionsVector[7] = 0
    if location[0] == width - 1:
        directionsVector[2] = directionsVector[4] = directionsVector[7] = 0
    rangeMax = sum(directionsVector)
    dirIndex = random.randint(0, rangeMax - 1)
    for i in range(0, 8):
        if directionsVector[i] == 0 and i <= dirIndex:
            dirIndex += 1
    if dirIndex == 0:
        return [location[0] - 1, location[1] - 1]
    elif dirIndex == 1:
        return [location[0], location[1] - 1]
    elif dirIndex == 2:
        return [location[0] + 1, location[1] - 1]
    elif dirIndex == 3:
        return [location[0] - 1, location[1]]
    elif dirIndex == 4:
        return [location[0] + 1, location[1]]
    elif dirIndex == 5:
        return [location[0] - 1, location[1] + 1]
    elif dirIndex == 6:
        return [location[0], location[1] + 1]
    elif dirIndex == 7:
        return [location[0] + 1, location[1] + 1]
    
def sobelFilter(location):
    surrounding = getSurroundingMatrix(location)
    sobelX = [
        [1, 2, 1],
        [0, 0, 0],
        [-1, -2, -1]
    ]
    sobelY = [
        [1, 0, -1],
        [2, 0, -2],
        [1, 0, -1]
    ]
    Gx = Gy = 0
    for i in range(0, len(surrounding)):
        for j in range(0, len(surrounding[0])):
            Gx += surrounding[i][j] * sobelX[i][j]
            Gy += surrounding[i][j] * sobelY[i][j]
    G = math.sqrt(Gx**2 + Gy**2)
    return G

def getNumberOfDirections(location):
    n = 0
    surrounding = getSurrounding(location)
    I = image[location[1]][location[0]]
    for i in surrounding:
        if abs(I - i) <= cT:
            n += 1
    return n

def getEdgesLocations(location):
    surrounding = getSurroundingMatrix(location)
    surroundingEdges = getSurroundingEdges(location)
    locations = []
    I = image[location[1]][location[0]]
    for y in range(0, len(surrounding)):
        for x in range(0, len(surrounding[0])):
            stdDev = getStdDev(location)
            if abs(I - surrounding[y][x]) <= cT and surroundingEdges[y][x] == 0 and surrounding[y][x] != -1 and stdDev >= 30 and not (x - 1 == location[0] and y - 1 == location[1]):
                locations.append([location[0] + x - 1, location[1] + y - 1]) 
    return locations

In [ ]:
class Manager(Agent):
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.foundLocations = []
        self.availableEdgeFollowers = []
        self.edgeFollowers = []
        self.scouts = []
        self.PASSWORD = "letsfindsomeedges"
    
    class Behav(CyclicBehaviour):
        
        async def on_start(self):
            numScouts = 30
            for i in range (1, numScouts + 1):
                agentId = "scout" + str(i) + "@localhost"
                location = [random.randint(0, width - 1), random.randint(0, height - 1)]
                a = Scout(agentId, self.agent.PASSWORD, location=location)
                await a.start()
                
            numFollowers = 50
            for i in range (1, numFollowers + 1):
                agentId = "edgeFollower" + str(i) + "@localhost"
                a = EdgeFollower(agentId, self.agent.PASSWORD)
                self.agent.availableEdgeFollowers.append(agentId)
                self.agent.edgeFollowers.append(a)
                await a.start()
            
        async def run(self):
            msg = None
            msg = await self.receive(timeout=3)
            if msg:
                intent = msg.get_metadata('intent')
                sender = msg.sender
                if intent == "edgeFound" or intent == "moreFollowersNeeded":
                    content = literal_eval(msg.body)
                    for i in content:
                        self.agent.foundLocations.append(i)
                    if len(self.agent.availableEdgeFollowers) > 0:
                        receiver = str(self.agent.availableEdgeFollowers[0])
                        location = str(self.agent.foundLocations[0])
                        msg = Message(
                        to=receiver,
                        body=location,
                        metadata={
                            'performative': 'inform',
                            'intent': 'newJobLocation'
                        })
                        self.agent.availableEdgeFollowers.pop(0)
                        self.agent.foundLocations.pop(0)
                        await self.send(msg)
                        print("-- " + str(self.agent.jid) + " -> " + receiver + ": job at " + location)
                elif intent == "followerAvailable":
                    content = msg.body
                    if len(self.agent.foundLocations) > 0:
                        receiver = str(sender)
                        location = str(self.agent.foundLocations[0])
                        msg = Message(
                        to=receiver,
                        body=location,
                        metadata={
                            'performative': 'inform',
                            'intent': 'newJobLocation'
                        })
                        self.agent.foundLocations.pop(0)
                        await self.send(msg)
                        print("-- " + str(self.agent.jid) + " -> " + receiver + ": job at " + location)
                    else:
                        self.agent.availableEdgeFollowers.append(sender)
                            
            else:
                print(str(self.agent.jid) + ": Timeout, stopping all agents!")
                for a in self.agent.edgeFollowers:
                    await a.stop()
                for a in self.agent.scouts:
                    await a.stop()
                self.kill(exit_code="Job finished")
            
        async def on_end(self):
            print("Behaviour of " + str(self.agent.jid) + " finished with exit code " + str(self.exit_code))
            
    async def setup(self):
        b = self.Behav()
        self.add_behaviour(b)

In [ ]:
class Scout(Agent):
    
    def __init__(self, *args, location, **kwargs):
        super().__init__(*args, **kwargs)
        self.location = location
        self.sobelThreshold = random.randint(400, 600)
    
    class Behav(CyclicBehaviour):
        
        async def on_start(self):
            self.counter = 0
            print(str(self.agent.jid) + " starting!")
            
        async def run(self):
            self.counter += 1
            if sobelFilter(self.agent.location) >= self.agent.sobelThreshold:
                edges[self.agent.location[1]][self.agent.location[0]] = 255
                locations = getEdgesLocations(self.agent.location)
                if len(locations) > 0:
                    msg = Message(
                    to="manager@localhost",
                    body=str(locations),
                    metadata={
                        'performative': 'inform',
                        'intent': 'edgeFound'
                    })
                    await self.send(msg)
                    print("-- " + str(self.agent.jid) + " -> manager@localhost: found edges at " + str(locations) + "!")
                    
            self.agent.location = [random.randint(0, width - 1), random.randint(0, height - 1)]
                
            if self.counter == lD:
                self.kill(exit_code="Job finished")

        async def on_end(self):
            print("Behaviour of " + str(self.agent.jid) + " finished with exit code " + str(self.exit_code))
            
    async def setup(self):
        print("Agent " + str(self.jid) + " created")
        b = self.Behav()
        self.add_behaviour(b)

In [ ]:
class EdgeFollower(Agent):
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
    
    class Behav(CyclicBehaviour):
        
        async def on_start(self):
            print(str(self.agent.jid) + " starting!")
            msg = None
            msg = await self.receive(timeout=60)
            if msg:
                intent = msg.get_metadata("intent")
                content = literal_eval(msg.body)
                if intent == "newJobLocation":
                    self.agent.location = content
            
        async def run(self):
            edges[self.agent.location[1]][self.agent.location[0]] = 255
            locations = getEdgesLocations(self.agent.location)
            if len(locations) == 1:
                self.agent.location = locations[0]
            elif len(locations) > 1:
                otherLocations = str(locations[1:])
                print("-- " + str(self.agent.jid) + " -> manager@localhost: need more followers at " + otherLocations + "!")
                msg = Message(
                to ="manager@localhost",
                body=otherLocations,
                metadata={
                    'performative': 'inform',
                    'intent': 'moreFollowersNeeded'
                })
                await self.send(msg)
                self.agent.location = locations[0]
            else:
                print("-- " + str(self.agent.jid) + " -> manager@localhost: finished my edge!")
                msg = Message(
                to="manager@localhost",
                body="I have finished my edge!",
                metadata={
                    'performative': 'inform',
                    'intent': 'followerAvailable'
                })
                await self.send(msg)
                
                msg = None
                msg = await self.receive(timeout=60)
                if msg:
                    intent = msg.get_metadata("intent")
                    content = literal_eval(msg.body)
                    if intent == "newJobLocation":
                        self.agent.location = content
                else:
                    self.kill(exit_code="Follower without job")
                
        async def on_end(self):
            print("Behaviour of " + str(self.agent.jid) + " finished with exit code " + str(self.exit_code))
            
    async def setup(self):
        print("Agent " + str(self.jid) + " created")
        b = self.Behav()
        self.add_behaviour(b)

In [ ]:
a = Manager("manager@localhost", "letsfindsomeedges")
a.start()

In [ ]:
a.stop()
plt.imshow(edges, cmap=plt.cm.binary)